# Table of Contents
 <p><div class="lev3 toc-item"><a href="#Clear-train-and-test-first-step-and-save-the-results" data-toc-modified-id="Clear-train-and-test-first-step-and-save-the-results-001"><span class="toc-item-num">0.0.1&nbsp;&nbsp;</span>Clear train and test first step and save the results</a></div><div class="lev3 toc-item"><a href="#Create-the-tagged-phrases" data-toc-modified-id="Create-the-tagged-phrases-002"><span class="toc-item-num">0.0.2&nbsp;&nbsp;</span>Create the tagged phrases</a></div><div class="lev4 toc-item"><a href="#add-tagger-to-dataframe" data-toc-modified-id="add-tagger-to-dataframe-0021"><span class="toc-item-num">0.0.2.1&nbsp;&nbsp;</span>add tagger to dataframe</a></div><div class="lev3 toc-item"><a href="#Create-data-for-cluster" data-toc-modified-id="Create-data-for-cluster-003"><span class="toc-item-num">0.0.3&nbsp;&nbsp;</span>Create data for cluster</a></div><div class="lev3 toc-item"><a href="#Variables-with-tagger" data-toc-modified-id="Variables-with-tagger-004"><span class="toc-item-num">0.0.4&nbsp;&nbsp;</span>Variables with tagger</a></div><div class="lev3 toc-item"><a href="#Create-test/train-with-clear-phrase" data-toc-modified-id="Create-test/train-with-clear-phrase-005"><span class="toc-item-num">0.0.5&nbsp;&nbsp;</span>Create test/train with clear phrase</a></div>

In [2]:
import ourfunctions as f
from time import time
import gc
import numpy as np  # linear algebra
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
import re
from gensim.models import word2vec

/home/ale/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [91]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [299]:
train_df = pd.read_csv("./train.csv", low_memory=True) # chunksize=)
test_df = pd.read_csv("./test.csv", low_memory=True)#, chunksize=)

print(train_df.shape)
print(test_df.shape)

## adjusting the nan value
train_df.fillna("", inplace=True)
test_df.fillna("", inplace=True)
train_df.info()

(404290, 6)
(2345796, 3)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
id              404290 non-null int64
qid1            404290 non-null int64
qid2            404290 non-null int64
question1       404290 non-null object
question2       404290 non-null object
is_duplicate    404290 non-null int64
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [3]:
question1, question2 = 'question1', 'question2'

### Clear train and test first step and save the results

In [ ]:
# first clear
f.add_clear_first(train_df, question1)
f.add_clear_first(train_df, question2)

In [ ]:
f.add_clear_first(test_df, question1)
f.add_clear_first(test_df, question2)

In [37]:
from joblib import Parallel, delayed
import multiprocessing
from itertools import chain


def clear_first_many(t):
    return [f.clear_first(x) for x in t]


def parallel_clear(df, q):
    step = min(len(df) // 100, 10000)
    start = df.index[0]
    stop = df.index[-1]
    num_cores = multiprocessing.cpu_count()
    return Parallel(n_jobs=num_cores)(
        delayed(clear_first_many)(df.loc[i:i + step-1][q].values)
        for i in range(start, stop, step))


def flatmap(f, items):
    return chain.from_iterable(map(f, items))


def add_clear1(df, q):
    l = flatmap(lambda x: x, parallel_clear(df, q))
    myl = [x for x in l]
    df[q + '_clear1'] = myl

In [38]:
add_clear1(train_df, question1)

In [39]:
train_df

,id,qid1,qid2,question1,question2,is_duplicate,question1_clear1
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,What is the step by step guide to invest in sh...
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,What is the story of Kohinoor Koh-i-Noor Diamond?
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,How can I increase the speed of my internet co...
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,Why am I mentally very lonely? How can I solve...
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,"Which one dissolve in water quickly sugar, sal..."
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1,Astrology: I am a Capricorn Sun Cap moon and c...
6,6,13,14,Should I buy tiago?,What keeps childern active and far from phone ...,0,Should I buy tiago?
7,7,15,16,How can I be a good geologist?,What should I do to be a great geologist?,1,How can I be a good geologist?
8,8,17,18,When do you use シ instead of し?,"When do you use ""&"" instead of ""and""?",0,When do you use シ instead of し?
9,9,19,20,Motorola (company): Can I hack my Charter Moto...,How do I hack Motorola DCX3400 for free internet?,0,Motorola company : Can I hack my Charter Motor...


In [40]:
add_clear1(train_df, question2)

In [114]:
test_df

,test_id,question1,question2,question1_clear1
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,How does the Surface Pro himself 4 compare wit...
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,Should I have a hair transplant at age 24? How...
2,2,What but is the best way to send money from Ch...,What you send money to China?,What but is the best way to send money from Ch...
3,3,Which food not emulsifiers?,What foods fibre?,Which food not emulsifiers?
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,How aberystwyth start reading?
5,5,How are the two wheeler insurance from Bharti ...,I admire I am considering of buying insurance ...,How are the two wheeler insurance from Bharti ...
6,6,How can I reduce my belly fat through a diet?,How can I reduce my lower belly fat in one month?,How can I reduce my belly fat through a diet?
7,7,"By scrapping the 500 and 1000 rupee notes, how...",How will the recent move to declare 500 and 10...,"By scrapping the 500 and 1000 rupee notes, how..."
8,8,What are the how best books of all time?,What are some of the military history books of...,What are the how best books of all time?
9,9,After 12th years old boy and I had sex with a ...,Can a 14 old guy date a 12 year old girl?,After 12th years old boy and I had sex with a ...


### Create the tagged phrases

In [9]:
unique_questions = f.create_unique_questions(train_df, test_df)

In [12]:
l = [ x for x in unique_questions['questions'].values ]

KeyboardInterrupt: 

In [ ]:
unique_questions['questions_clear_1'] = pd.Series(l)

In [ ]:
f.create_tagger_csv(unique_questions, 2000)

In [ ]:
start_stop = [
    [30000, 32001],
    [46000, 48001],
    [54000, 56001],
    [158000, 160001],
    [248000, 250001],
    [258000, 260001],
    [342000, 344001],
    [358000, 360001],
    [390000, 392001],
    [396000, 398001],
    [432000, 434001],
    [440000, 442001],
    [460000, 462001],
    [544000, 546001],
    [558000, 560001],
    [574000, 576001],
    [696000, 698001],
    [756000, 758001],
    [848000, 850001],
    [890000, 892001],
    [958000, 960001],
    [970000, 972001],
    [1010000, 1012001],
]

In [ ]:
f.fix_creations_csv_tagger(start_top, 20)

#### add tagger to dataframe

In [5]:
start = time()
import os
mycsv = os.listdir('csv_question_tag/')
df = pd.concat([pd.read_csv('csv_question_tag/'+csv) for csv in mycsv]) 
stop = time()
print((stop - start)/60, 'minutes')

0.4400623877843221 minutes


In [6]:
df.drop('Unnamed: 0', axis=1,inplace=True)
df.columns = ['tagger', 'questions_clear_1']
df.drop_duplicates(inplace=True)

In [ ]:
list_tagger = df.set_index('questions_clear_1').to_dict()['tagger']
temp_dict = unique_questions.set_index('questions').to_dict()['questions_clear_1']

In [ ]:
del df
del unique_questions
gc.collect()

In [13]:
from functools import reduce

def recover_tagg_list(tag_string):
    l = [
        re.split(r'[\'\"],\s[\'\"]', t[2:-1]) 
        for t in tag_string[1:-2].split('], ')
    ]
    return l

def get_phrase_from_tagger(tagg_list):
    l = [
        c[2] if (c[2] != '<unknown>') and (c[2] != '@card@') else c[0]
        for c in tagg_list if len(c) == 3
    ]
    return reduce(lambda x, y: x + ' ' + y, l)

def convert_tag_to_phrase(df, question_after_tag, tagger):
    df[question_after_tag] = df.apply(
        axis=1, func=lambda x: get_phrase_from_tagger( recover_tagg_list(x[tagger])) )

In [4]:
df = pd.read_csv('tagg_list.csv')
df.dropna(inplace=True)

In [5]:
convert_tag_to_phrase(df, 'quest_tag', 'tagger')

In [12]:
def clear_second(text):
    text = f.clear_correzioni_mano(text)
    text = f.clear_text_second_step(text)
    text = f.remove_punctuations(text)
    text = f.remove_stopwors(text, f.my_stopwords)
    return text

In [13]:
df['quest_final'] = df.apply(axis=1, func=lambda x:clear_second(x['quest_tag']))

In [14]:
df.to_csv('risultato_tagger.csv')

### Create test/train with clear phrase

In [ ]:
df = pd.read_csv('risultato_tagger.csv', encoding='latin1')

In [314]:
convert = df.set_index('questions_clear_1').to_dict()['quest_final']

In [309]:
question1, question2 = 'question1', 'question2'
f.add_clear_first(train_df, question1)
f.add_clear_first(train_df, question2)

In [335]:
f.add_clear_first(test_df, question1)
f.add_clear_first(test_df, question2)

In [327]:
def get_final_quest(x):
    try:
        return convert[x]
    except KeyError:
        return 'ciccia'

In [332]:
train_df['question1_final'] = train_df.apply(axis=1, func=lambda x: get_final_quest(x['question1_clear_1']))
train_df['question2_final'] = train_df.apply(axis=1, func=lambda x: get_final_quest(x['question2_clear_1']))

In [336]:
test_df['question1_final'] = test_df.apply(axis=1, func=lambda x: get_final_quest(x['question1_clear_1']))
test_df['question2_final'] = test_df.apply(axis=1, func=lambda x: get_final_quest(x['question2_clear_1']))

In [338]:
train_df.to_csv('train_clear.csv')
test_df.to_csv('test_clear.csv')

In [ ]:
tagger_list = {
    'CC': 'Coordinating conjunction',
    'CD': 'Cardinal number',
    'DT': 'Determiner',
    'EX': 'Existential there',
    'FW': 'Foreign word',
    'IN': 'Preposition or subordinating conjunction',
    'JJ': 'Adjective',
    'JJR': 'Adjective, comparative',
    'JJS': 'Adjective, superlative',
    'LS': 'List item marker',
    'MD': 'Modal',
    'NN': 'Noun, singular or mass',
    'NNS': 'Noun, plural',
    'NP': 'Proper noun, singular',
    'NPS': 'Proper noun, plural',
    'PDT': 'Predeterminer',
    'POS': 'Possessive ending',
    'PP': 'Personal pronoun',
    'PP$': 'Possessive pronoun',
    'RB': 'Adverb',
    'RBR': 'Adverb, comparative',
    'RBS': 'Adverb, superlative',
    'RP': 'Particle',
    'SYM': 'Symbol',
    'TO': 'to',
    'UH': 'Interjection',
    'VB': 'Verb, base form',
    'VBD': 'Verb, past tense',
    'VBG': 'Verb, gerund or present participle',
    'VBN': 'Verb, past participle',
    'VBP': 'Verb, non-3rd person singular present',
    'VBZ': 'Verb, 3rd person singular present',
    'WDT': 'Wh-determiner',
    'WP': 'Wh-pronoun',
    'WP$': 'Possessive wh-pronoun',
    'WRB': 'Wh-adverb'
}